In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import os
import numpy as np
import sys
import pandas as pd
sys.path.append("/usr/local/lib/python3.5/dist-packages/")
import nibabel as nib

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim






class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        
        self.conv1 = nn.Conv3d(1, 6, 8, stride=2)
        self.conv2 = nn.Conv3d(6,16,3)
        
        #self.maxpool1 = F.max_pool3d(2)
        #self.relu1 = F.relu()
        
        #self.maxpool2 = F.max_pool3d(2)
        
        
        #self.conv1 = nn.Conv2d(1, 6, 3)
        #self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 21 * 25 * 21, 5)  # 6*6 from image dimension
        self.fc2 = nn.Linear(5, 20)
        self.fc3 = nn.Linear(20, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = self.conv1(x)
        #x = self.relu1(x)
        #x = self.max_pool3d1(x)
        
        #x = self.conv2(x)
        #x = self.relu1(x)
        #x = self.max_pool3d2(x)
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = F.max_pool3d(x)
        x = F.max_pool3d(F.relu(self.conv1(x)), (2, 2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool3d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

device = torch.device("cuda")
net.cuda()

optimizer = optim.SGD(net.parameters(), lr=0.005)
criterion = nn.BCELoss()
criterion.cuda()


batch_size = 20
D, R, C = (182, 218,182)
x = torch.zeros((batch_size,1,D,R,C))
y = torch.zeros(batch_size)
offset = 0



Net(
  (conv1): Conv3d(1, 6, kernel_size=(8, 8, 8), stride=(2, 2, 2))
  (conv2): Conv3d(6, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (fc1): Linear(in_features=176400, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
)


In [48]:
from tqdm import tqdm

In [ ]:
correct = 0
total = 0

    
df = np.genfromtxt("preprocessing/encoded_track2_labels_v2.csv",delimiter=',')
x_loc = df[1:,0]
y_loc = df[1:,1]



#df = np.genfromtxt("preprocessing/encoded_track2_labels.csv",delimiter=',')
#x_loc = df[1:,2]
#y_loc = df[1:,3]


epochs = 1500
# in your training loop:
for epoch in tqdm(range(epochs)):
    for i in range(150):
        for j in range(batch_size):
            loc = x_loc[j+offset]
            example_filename = "../brainhack/Sub-" + str(int(loc)) + ".nii.gz"
            img = nib.load(example_filename)
            data = img.get_fdata()
            x[j,0,:,:,:] = torch.from_numpy(data).float()
            y[j] = y_loc[j+offset]
        offset = j+offset
        input = x/x.max()
        
        
        
        optimizer.zero_grad()   # zero the gradient buffers
        input = input.cuda()
        y = y.cuda()
        output = net(input)
        loss = criterion(output.view(batch_size), y.view(batch_size))
        loss.backward()
        optimizer.step()    # Does the update
        correct += ((output > 0.5).view(-1)*(y>0.5).view(-1)).sum()
        total += batch_size
        print(correct.float()*100/total)
    
    # save model
    if epoch % 10 == 0:
        torch.save(net,'cnn_epoch_'+str(epoch)+'.pt')
#         print(loss)
#         print(output)
#         print(y)
#         print(input.sum())
        











  0%|          | 0/1500 [00:00<?, ?it/s]

tensor(70., device='cuda:0')
tensor(62.5000, device='cuda:0')
tensor(61.6667, device='cuda:0')
tensor(62.5000, device='cuda:0')
tensor(62., device='cuda:0')
tensor(63.3333, device='cuda:0')
tensor(64.2857, device='cuda:0')
tensor(66.2500, device='cuda:0')
tensor(68.8889, device='cuda:0')
tensor(68.5000, device='cuda:0')
tensor(69.0909, device='cuda:0')
tensor(67.5000, device='cuda:0')
tensor(67.3077, device='cuda:0')
tensor(67.8571, device='cuda:0')
tensor(69.3333, device='cuda:0')
tensor(68.4375, device='cuda:0')
tensor(68.2353, device='cuda:0')
tensor(68.0556, device='cuda:0')
tensor(69.2105, device='cuda:0')
tensor(69.5000, device='cuda:0')
tensor(69.2857, device='cuda:0')
tensor(70.2273, device='cuda:0')
tensor(71.3043, device='cuda:0')
tensor(71.4583, device='cuda:0')
tensor(71.4000, device='cuda:0')
tensor(71.5385, device='cuda:0')
tensor(71.6667, device='cuda:0')
tensor(71.7857, device='cuda:0')
tensor(71.0345, device='cuda:0')
tensor(71., device='cuda:0')
tensor(70.6452, device

In [ ]:
offset = 0

In [21]:
 ((output > 0.5).view(-1)*(y>0.5).view(-1)).sum()

tensor(13, device='cuda:0')

In [ ]:
!sudo -H pip3 install nibabel